In [1]:
import pandas as pd
import numpy as np
import json
import dask.bag as db
import dask.dataframe as dd
import tensorflow as tf
#import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

2024-08-27 10:54:24.372632: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Table of Contents
[Step 0](#part0) The statement of the problem <br>
[Step 1](#part1) From bres select impressions with sflr == 15 <br>
[Step 2](#part2) Extract relevant fields from the raw bres  with sflr == 15: resulting file 'processed_bres_15_date.pkl' <br>
[Step 3](#part3) Extract relevant fields form om file. Mark those impressions in bres that are in om with Y or N depending on whether the bid was successful or not <br>
[Step 4](#part4) Plot the data, zero in on the floor price bids  <br>
[Step 5](#part5) Clean the resulting dataframe
    a. convert time to hourly slots
    b. cleanup the text /categories in the columns <br>
[Step 6](#part6) Split the dataframe into two:  sbpr = 15 and sbpr != 15 <br>
[Step 7](#part7). Use sbpr = 15 for classification. <br>
[Step 8](#part8) Based on the model in Step 6 predict which impressions should not be in sbpr != 15 dataframe. Eliminate those from the dataframe <br>
[Step 9](#part9) Plot the learned probability.

Step 0: The statement of the problem and summary of the results. <br>
<a id="part0"></a>

The problem comes from the world of real-time bidding in advertising. To simplify for the purposes of this short POC, a platform submits bids for ads (impressions), following some strategy. Some of these bids get accepted and the question is how to optimize the strategy. 
More specifically, each impression is submitted with a floor price 'sflr' and the platform places bids at prices 'sbpr' $>=$ 'sflr'. In the ideal world one expects the probability of the bid being accepted to be a monotonically increasing function the bid price 'sbpr'. 
However, as the plot below shows this is totally not the case. In this example I'm displaying the probability for a single 'sflr'$=$15.0 for one day.

In [3]:
# insert figure

Several things jump out from the figure:
1. there are lots of bids at the floor price.
2. there are bid prices with 100% probability of winning.
to be precise 

In [4]:
df_ex0409_15= pd.read_pickle("bres_0409_15_clean")

In [5]:
df_ex0409_15.head()

,sbpr,time,id,minduration,maxduration,minbitrate,maxbitrate,app_id,app_name,app_bundle,app_pub_id,genre,language,device_make,device_model,device_os,geo_region,geo_metro
0,15.0,0,51821639-48a2-447b-b5de-aee0beb878bd,1,120,1,280000,702131,plex,g15147002586,1034,entertainment,NaN,samsung,tizen tv,tizen,sc,546
1,15.0,0,5aa54938-a3a7-447b-b6cf-14b43764f8e3,1,120,1,280000,711665,"sling: live tv, shows & movies",com.sling,1034,NaN,NaN,generic,generic android tv,android,oh,NaN
2,15.0,0,37b89a1e-e9e3-4697-a7f9-e20f0906fd30,1,120,1,280000,726329,plex,g15147002586,1034,entertainment,NaN,samsung,tizen tv,tizen,tn,691
3,16.5,0,ee6d522b-838e-4ecd-bb10-71637c4b0505,1,70,1,280000,711667,fubotv: watch live sports & tv,905401434,1034,NaN,en,apple,tv a1625,other,ga,524
4,15.0,0,97253b20-00e5-445c-90c0-8cdf19da12c6,1,70,1,280000,711667,fubotv: watch live sports & tv,905401434,1034,NaN,en,apple,tv a1625,other,ga,524
